# Jason's Honours Project
## Take 4
Attempt to optimise with cooling

In [1]:
# Libraries
import numpy as np
from scipy.constants import mu_0
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from PIL import Image, ImageDraw, ImageColor
import matplotlib.pyplot as plt
from scipy.optimize import minimize, NonlinearConstraint, Bounds

In [12]:
# Relative constants
# Things that aren't easily changed
BR_CORE = 1.2
SIGMA_WIRE = 2.3e6
V_MAX = 12
L_TUBE_MAX = 4.5
L_PLUNGER = 0.005
THICKNESS_SHELL = 0.0025
DISTANCE_RG2 = 0.001
REL_PERM_STEEL = 2000
REL_PERM_NEO = 1.05
REL_PERM_AIR = 1

moe = 1e-6

$r_{shellIn} = r_{core}+r_{wireOut}\cdot layers$ Shell inner radius

$\Delta heat = \rho c_p Q \Delta T$ Heat power difference of liquid metal start and end of motor

$R_{total} = (\frac{l_{wireTotal}}{\sigma \pi {r_{wireIn}}^2})^2$

$P_{mechanical} = 2d_{drive}F_{motor}f$

$rg_1 = \frac{ln(\frac{r_{shellIn}}{r_{core}})}{2\pi l_{connector}}$

$rg_2 = \frac{ln(\frac{r_{shellIn}}{r_{plunger}})}{2\pi l_{plunger}}$

$rm_{ttl} = rg_1 + rg_2$

$emmf = B_r * l_{core}$

$\phi = \frac{emmf}{rm_{ttl}}$

$A_{action} = 2\pi l_{connector}(r_{core}+2layers\cdot r_{wireOut}-r_{wireOut})$ 

$B = \frac{\phi}{A_{action}}$

$ I=\frac{\Delta heat}{{R_{total}}^2 - B l_{wireInField}}$

Optimise for minimum power ($I\sqrt{\frac{l_{wireTotal}}{\pi r_{wireIn}^2 \rho}}$):
1. Given force ($ F_{motor} $), drive distance ($ d_{drive} $) and frequency ($f$) 9N - 
2. Given magnet strength ($ B_r $)
3. Maximum volume of liquid metal in circuit ($ A_{wire} l_{wireTotal} $)
4. Maximum shell dimensions ($ r_{shellIn} $)
5. Maximum core length ($ l_{core} $)
6. Drive distance doesn't exceed motor dimensions ($ d_{drive}  < \frac{l_{wireTotal} - l_{wireInField}}{2\pi r_{shellIn}} $)
7. Inner motor parts don't exceed outer motor parts ($ r_{shellIn} >  r_{plunger}$, $ r_{shellIn} > r_{core}$, $ r_{plunger} >= r_{core}$)
8. Maximum temperature change ($ \Delta T $)
9. Total length of wire is larger than wire in field ($ l_{wireTotal} > l_{wireInField} $)
10. Minimum wire wall thickness ($ r_{wireOut} - r_{wireIn} > 0.5 $)
11. Length of total wire fills a whole layer ($ l_{wireTotal} = 2\pi l_{core}(layers\cdot r_{core}+r_{wireOut}\frac{(1+layers)layers}{2}$)
12. Layers is an integer

Notes:
- Q is given to Simran as 0.5 ml/s

In [4]:
def motorMain(x,rho,sigma,cp,dDrive,fMotor,freq,br):
    lWireTotal = x[0]
    lWireInField = x[1]
    rWireIn = x[2]
    rWireOut = x[3]
    lCore = x[4]
    rCore = x[5]
    lConnector = x[6]
    lPlunger = x[7]
    rPlunger = x[8]
    rShellIn = x[9]
    layers = x[10]
    Q = x[11]
    deltaT = x[12]
    
    rShellIn = rCore+rWireOut*layers # Shell inner radius
    
    deltaHeat = rho*Q*cp*deltaT # Heat power difference of liquid metal start and end of motor
    
    rTotal = lWireTotal/np.pi/rWireIn**2/sigma # Total wire resistance
    
    pMechanical = 2*dDrive*fMotor*freq # Mechanical power output
    
    rg1 = np.log(rShellIn/rCore)/2*np.pi*lConnector
    
    rg2 = np.log(rShellIn/rPlunger)/2*np.pi*lPlunger
    
    rmTtl = rg1+rg2
    
    emmf = br*lCore
    
    flux = emmf / rmTtl
    
    area_action = 2*np.pi*lConnector*(rCore+2*layers*rWireOut-rWireOut)
    
    B = flux / area_action
    
    I = deltaHeat/(rTotal**2 - B*lWireInField)
    
    power = I * np.sqrt(lWireTotal/(np.pi*rho*rWireIn**2))
                         
    return power

In [ ]:
def constraintsFun(x):
    l = []
    # 3. Maximum volume of liquid metal in circuit ($ A_{wire} l_{wireTotal} $)
    aWire = rWireIn**2*np.pi
    l.append(aWire*lWireTotal)
    
    # 4. Maximum shell dimensions ($ r_{shellIn} $)
    l.append(rShellIn)
    
    # 5. Maximum core length ($ l_{core} $)
    l.append(lCore)
    
    # 6. Drive distance doesn't exceed motor dimensions ($ d_{drive}  < \frac{l_{wireTotal} - l_{wireInField}}{2\pi r_{shellIn}} $)
    drive_distance_dimension_check = (lWireTotal-lWireInField)/(2*np.pi*rShellIn)
    l.append(drive_distance_diemsnion_check)
    # This needs to be less than dDrive
    
    #7. Inner motor parts don't exceed outer motor parts ($ r_{shellIn} >  r_{plunger}$, $ r_{shellIn} > r_{core}$, $ r_{plunger} >= r_{core}$)
    plunger_dimension_check = rShellIn-rPlunger
    core_dimension_check = rShellIn-rCore
    plunger_core_dimension_check = rPlunger-rCore
    # All three need to be larger than 0
    
    l.append(plunger_dimension_check)
    l.append(core_dimension_check)
    l.append(plunger_core_dimension_check)
    
    #8. Maximum temperature change ($ \Delta T $)
    # Needs to be larger than value
    l.append(deltaT)
    
    #9. Total length of wire is larger than wire in field ($ l_{wireTotal} > l_{wireInField} $)
    # Needs to be larger than 0
    wire_length_check = lWireTotal - lWireInField
    l.append(wire_length_check)
    
    #10. Minimum wire wall thickness ($ r_{wireOut} - r_{wireIn} > 0.5 $)
    # Needs to be larger than value
    wall_thickness_check = rWireOut - rWireIn
    l.append(wall_thickness_check)
    
    #11. Length of total wire fills a whole layer ($ l_{wireTotal} = 2\pi l_{core}(layers\cdot r_{core}+r_{wireOut}\frac{(1+layers)layers}{2}$)
    # Needs to be 0
    whole_layer_check = lWireTotal - 2*np.pi*lCore*layers*(rCore+rWireOut*(1+layers)/2)
    l.append(whole_layer_check)
    
    #12. Layers is an integer
    # if integer, =1, else =0
    # needs to be 1
    if np.abs(int(layers) - layers) > moe:
        l.append(0)
    else:
        l.append(1)

| x | Name | Units | Description |
| - |------|-------|-------------|
| 0 | lWireTotal | m | Total length of silicone tube |
| 1 | lWireInField | m | Length of silicone tube in field |
| 2 | rWireIn | m | Inner radius of silicone tube |
| 3 | rWireOut | m | Outer radius of silicone tube |
| 4 | lCore | m | Length of motor core |
| 5 | rCore | m | Radius of motor core |
| 6 | lConnector | m | How much magnetic connector at the end of the magnet |
| 7 | lPlunger | m | Length of plunger connector |
| 8 | rPlunger | m | Plunger diameter |
| 9 | layers | #NA# | Number of layers of wire |
| 10 | Q | m^3/s | Flow rate of liquid metal |
| 11 | deltaT | degC | Change in temperature of liquid metal before and after motor |

In [1]:
# Graphing